In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from os import listdir

from torchvision import transforms, utils
from skimage import io, transform
from torchvision import transforms 

import pandas as pd 
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
import glob

In [3]:
df=pd.read_csv('data/artists.csv')

In [5]:
df.head()

,id,name,years,genre,nationality,bio,wikipedia,paintings
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194


In [6]:
len(df.name)

50

In [10]:
mypath = 'data/resized'

In [11]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [12]:
onlyfiles

['Albrecht_DuÔòá├¬rer_1.jpg',
 'Albrecht_DuÔòá├¬rer_10.jpg',
 'Albrecht_DuÔòá├¬rer_100.jpg',
 'Albrecht_DuÔòá├¬rer_101.jpg',
 'Albrecht_DuÔòá├¬rer_102.jpg',
 'Albrecht_DuÔòá├¬rer_103.jpg',
 'Albrecht_DuÔòá├¬rer_104.jpg',
 'Albrecht_DuÔòá├¬rer_105.jpg',
 'Albrecht_DuÔòá├¬rer_106.jpg',
 'Albrecht_DuÔòá├¬rer_107.jpg',
 'Albrecht_DuÔòá├¬rer_108.jpg',
 'Albrecht_DuÔòá├¬rer_109.jpg',
 'Albrecht_DuÔòá├¬rer_11.jpg',
 'Albrecht_DuÔòá├¬rer_110.jpg',
 'Albrecht_DuÔòá├¬rer_111.jpg',
 'Albrecht_DuÔòá├¬rer_112.jpg',
 'Albrecht_DuÔòá├¬rer_113.jpg',
 'Albrecht_DuÔòá├¬rer_114.jpg',
 'Albrecht_DuÔòá├¬rer_115.jpg',
 'Albrecht_DuÔòá├¬rer_116.jpg',
 'Albrecht_DuÔòá├¬rer_117.jpg',
 'Albrecht_DuÔòá├¬rer_118.jpg',
 'Albrecht_DuÔòá├¬rer_119.jpg',
 'Albrecht_DuÔòá├¬rer_12.jpg',
 'Albrecht_DuÔòá├¬rer_120.jpg',
 'Albrecht_DuÔòá├¬rer_121.jpg',
 'Albrecht_DuÔòá├¬rer_122.jpg',
 'Albrecht_DuÔòá├¬rer_123.jpg',
 'Albrecht_DuÔòá├¬rer_124.jpg',
 'Albrecht_DuÔòá├¬rer_125.jpg',
 'Albrecht_DuÔòá├¬rer_126.jpg',
 'Albrecht_Du

In [13]:
import re
def get_numbers_from_filename(filename):
    return re.search(r'\d+', filename).group(0)

In [14]:
num_list=[]
for filename in onlyfiles:
    num_list.append(get_numbers_from_filename(filename))

In [15]:
num_list

['1',
 '10',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '11',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '13',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '14',
 '140',
 '141',
 '142',
 '143',
 '144',
 '145',
 '146',
 '147',
 '148',
 '149',
 '15',
 '150',
 '151',
 '152',
 '153',
 '154',
 '155',
 '156',
 '157',
 '158',
 '159',
 '16',
 '160',
 '161',
 '162',
 '163',
 '164',
 '165',
 '166',
 '167',
 '168',
 '169',
 '17',
 '170',
 '171',
 '172',
 '173',
 '174',
 '175',
 '176',
 '177',
 '178',
 '179',
 '18',
 '180',
 '181',
 '182',
 '183',
 '184',
 '185',
 '186',
 '187',
 '188',
 '189',
 '19',
 '190',
 '191',
 '192',
 '193',
 '194',
 '195',
 '196',
 '197',
 '198',
 '199',
 '2',
 '20',
 '200',
 '201',
 '202',
 '203',
 '204',
 '205',
 '206',
 '207',
 '208',
 '209',
 '21',
 '210',
 '211',
 '212',


In [16]:
data_list=[]
for name in tqdm(df.name.tolist()):
    for file in onlyfiles:
        if all(x in file for x in name.split(' ')):
            data_list.append([name,get_numbers_from_filename(file),f'{mypath}/{file}'])
  

100%|██████████| 50/50 [00:00<00:00, 273.75it/s]


In [17]:
df_bilder=pd.DataFrame(data=data_list,columns=['name','num','path'])

In [18]:
df=df_bilder.merge(df,on='name')

In [19]:
y=df.name.tolist()
X=df

In [20]:
random_seed=42
test_size=0.3

X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, random_state=random_seed, test_size=test_size)
